In [155]:
import numpy as np
import pandas as pd
import seaborn as sns

In [156]:
path = r"E:\Metro1\artifacts\\"

access_df=pd.read_csv(path+"access_data.csv")
egress_df=pd.read_csv(path+"egress_data.csv")



In [157]:
# remove duplicate rows with same id column from dataframe
access_df.drop_duplicates(subset ="ID",inplace = True)
egress_df.drop_duplicates(subset ="ID",inplace = True)

print(access_df.shape,egress_df.shape)

# print(access_df.shape,egress_df.shape)
access_df['RENT_BICYCLE'].value_counts()

(831, 74) (715, 76)


RENT_BICYCLE
4    388
3    233
1    161
2     49
Name: count, dtype: int64

In [158]:
dict={'Walk ':'Walk', 'Auto Rickshaw ':'Ride hire', 'Dropped-off by friends/others ':'Ride hire',
       'Two-Wheeler ':'Personal vehicle', 'Ola/Uber car ':'Personal vehicle', 'Bicycle ':'Personal vehicle', 'Car ':'Personal vehicle',
       'Rapido/Uber moto/Ola App two-wheeler ':'Ride hire', 'Shuttle':'Ride hire',
       'Rapido/uber moto/Ola App two-wheeler ':'Ride hire', 'Uber auto ':'Ride hire',
       'Company shuttle':'Ride hire', 'Office metro feeder':'Ride hire',
       'Dropped-off by friends/family members/others ':'Ride hire',
       'Drove and parked a two-Wheeler ':'Personal vehicle', 'Drove and parked a car ':'Personal vehicle',
       'Friends two wheeler':'Personal vehicle'}

access_df['ACCESS_MODE']=access_df['ACCESS_MODE'].map(dict)

In [159]:
access_df['AGE'].unique()

age_mapper={'> 60 yrs ':3, '18-25 yrs ':1, '26-35 yrs ':1, '18-25yrs ':1, '26-35yrs ':1,
       '46-60 yrs ':3, '36-45 yrs ':2, '36-45yrs ':2,'46-60yrs ':3,'> 60 yrs ':3, '18-25 yrs ':1, '26-35 yrs ':1, '26-35yrs ':1, '18-25yrs ':1,
       '46-60 yrs ':3, '36-45 yrs ':2, '36-45yrs ':2, '46-60yrs ':3}
access_df['AGE_C']=access_df['AGE'].map(age_mapper)
egress_df['AGE_C']=egress_df['AGE'].map(age_mapper)

print(access_df['AGE_C'].value_counts(),egress_df['AGE_C'].value_counts())

access_df['AGE1']=0
access_df['AGE2']=0
access_df['AGE3']=0

# if AGE_C is 1 then AGE1 is 1
access_df.loc[access_df['AGE_C']==1,'AGE1']=1
# if AGE_C is 2 then AGE2 is 1
access_df.loc[access_df['AGE_C']==2,'AGE2']=1
# if AGE_C is 3 then AGE3 is 1
access_df.loc[access_df['AGE_C']==3,'AGE3']=1

AGE_C
1    660
2    108
3     63
Name: count, dtype: int64 AGE_C
1    577
2     81
3     57
Name: count, dtype: int64


In [160]:
access_df.columns

# rename few columns
access_df.rename(columns = {'A_DISTANCE_MAP':'DISTANCE', 'A_TIME_MAP':'TIME','ACCESS_SLOPE':'SLOPE',
                            'E_SLOPE_TYPE':'SLOPE_TYPE'}, inplace = True)

In [161]:
access_df['UP']=0
access_df['DOWN']=0
access_df['FLAT']=0

# if SLOPE is positive then make POS as 1 else 0
access_df.loc[access_df['SLOPE']>1,'UP']=1
access_df.loc[access_df['SLOPE']<-1,'DOWN']=1
access_df.loc[(access_df['SLOPE']>=-1) & (access_df['SLOPE']<=1),'FLAT']=1

access_df['VEH']=0

access_df['POS']=0

# if slope is greater than 0 then POS is 1 else 0
access_df.loc[access_df['SLOPE']>0,'POS']=1

# if CARS or TWO_WHEELER is greater than 1 then mark VEH as 1 else 0
access_df.loc[(access_df['CARS']>1) | (access_df['TWO_WHEELER']>1),'VEH']=1


access_df['SHORT']=0
access_df['MEDIUM']=0
access_df['LONG']=0

# if DISTANCE_MAP is less than 1 then make LOW as 1
access_df.loc[access_df['DISTANCE']<1,'SHORT']=1
# if DISTANCE_MAP is greater than 1 and less than 3 then make MEDIUM 1
access_df.loc[(access_df['DISTANCE']>=1) & (access_df['DISTANCE']<3),'MEDIUM']=1
# if DISTANCE_MAP is greater than 3 then make HIGH as 1
access_df.loc[access_df['DISTANCE']>=3,'LONG']=1

access_df['USE']=1
access_df.loc[(access_df['RENT_BICYCLE']==4) | (access_df['RENT_BICYCLE']==2),'USE']=0

In [162]:
# convert 2 in RENT_BICYCLE to 4 and 3 to 1 in access_df dataframe
access_df['RENT_BICYCLE'] = access_df['RENT_BICYCLE'].replace({4: 2, 3: 1})

# create a new column WORK where if O_TYPE or D_TYPE is work then it is 1 else 0
access_df['WORK']=0
access_df.loc[(access_df['O_TYPE']=='Work') | (access_df['D_TYPE']=='Work'),'WORK']=1

access_df['MODE']=0


access_df['DEN']=0
access_df['DEN']=access_df['A_INTERSECTIONS']/access_df['DISTANCE']

access_df=access_df[access_df['DEN']<30]

access_df['UNO']=1

access_df['HIGH']=0
access_df['MED']=0

# if DEN is greater than 66 percentile of data of DEN then HIGH is 1 for that entry
# if DEN is between 33 and 66 percentiles of data of DEN then MED is 1 for that entry 
access_df.loc[access_df['DEN']>access_df['DEN'].quantile(0.66),'HIGH']=1
access_df.loc[(access_df['DEN']<=access_df['DEN'].quantile(0.66)) & (access_df['DEN']>access_df['DEN'].quantile(0.33)),'MED']=1

access_df['DIST1']=0
access_df['DIST2']=0
# access_df['DIST3']=0
access_df['DIST4']=0
access_df['DIST5']=0

access_df.loc[access_df['DISTANCE']<1,'DIST1']=1
access_df.loc[(access_df['DISTANCE']>=1) & (access_df['DISTANCE']<2),'DIST2']=1
access_df.loc[(access_df['DISTANCE']>=2) & (access_df['DISTANCE']<3),'DIST2']=1
access_df.loc[(access_df['DISTANCE']>=3) & (access_df['DISTANCE']<4),'DIST4']=1
access_df.loc[(access_df['DISTANCE']>=4),'DIST5']=1


# output columns ID, RENT_BICYCLE,MALE, ACCESS_SLOPE, A_SLOPE_TYPE, A_DISTANCE_MAP, A_TIME_MAP,O_TYPE,D_TYPE,ACCESS_MODE,AGE_C,N_BICYCLES,CARS,
# TWO_WHEELER, OTHERS, A_INTERSECTINS from access_df dataframe to csv file
access_df[['ID', 'RENT_BICYCLE', 'MALE', 'SLOPE', 'SLOPE_TYPE', 'DISTANCE', 'TIME','O_TYPE','D_TYPE','ACCESS_MODE',
           'AGE1','AGE2','AGE3','VEH','WORK','DEN','UNO','SHORT','LONG','MEDIUM','UP','DOWN','FLAT','POS','HIGH','MED',
        'DIST1','DIST2','DIST4','DIST5']].to_csv(path+"access_data_analysis.csv",index=False)

KeyError: "['DIST3'] not in index"

In [ ]:
print(access_df[access_df['DISTANCE']<1].shape[0])
print(access_df[(access_df['DISTANCE']>=1) & (access_df['DISTANCE']<2)].shape[0])
print(access_df[(access_df['DISTANCE']>=2) & (access_df['DISTANCE']<3)].shape[0])
print(access_df[(access_df['DISTANCE']>=3) & (access_df['DISTANCE']<4)].shape[0])
print(access_df[(access_df['DISTANCE']>=4)].shape[0])

248
203
141
68
115


In [ ]:
# lets get the value counts of ACCESS_MODE for each unique entry of RENT_BICYCLE
access_df.groupby(['RENT_BICYCLE'])['ACCESS_MODE'].value_counts()

RENT_BICYCLE  ACCESS_MODE     
1             Walk                167
              Personal vehicle    104
              Ride hire           103
2             Walk                217
              Ride hire            94
              Personal vehicle     90
Name: count, dtype: int64

In [ ]:
# rename few columns
egress_df.rename(columns = {'E_DISTANCE_MAP':'DISTANCE', 'E_TIME_MAP':'TIME','EGRESS_SLOPE':'SLOPE','A_INTERSECTIONS':'INTERSECTIONS'
                            ,'E_SLOPE_TYPE':'SLOPE_TYPE'}, inplace = True)

In [ ]:
egress_df['UP']=0
egress_df['DOWN']=0
egress_df['FLAT']=0

# if SLOPE is positive then make POS as 1 else 0
egress_df.loc[egress_df['SLOPE']>1,'UP']=1
egress_df.loc[egress_df['SLOPE']<-1,'DOWN']=1
egress_df.loc[(egress_df['SLOPE']>=-1) & (egress_df['SLOPE']<=1),'FLAT']=1

egress_df['VEH']=0

egress_df['POS']=0

# if slope is greater than 0 then POS is 1 else 0
egress_df.loc[egress_df['SLOPE']>0,'POS']=1

# if CARS or TWO_WHEELER is greater than 1 then mark VEH as 1 else 0
egress_df.loc[(egress_df['CARS']>1) | (egress_df['TWO_WHEELER']>1),'VEH']=1


egress_df['SHORT']=0
egress_df['MEDIUM']=0
egress_df['LONG']=0

# if DISTANCE_MAP is less than 1 then make LOW as 1
egress_df.loc[egress_df['DISTANCE']<1,'SHORT']=1
# if DISTANCE_MAP is greater than 1 and less than 3 then make MEDIUM 1
egress_df.loc[(egress_df['DISTANCE']>=1) & (egress_df['DISTANCE']<3),'MEDIUM']=1
# if DISTANCE_MAP is greater than 3 then make HIGH as 1
egress_df.loc[egress_df['DISTANCE']>=3,'LONG']=1


egress_df['AGE1']=0
egress_df['AGE2']=0
egress_df['AGE3']=0 

# if AGE_C is 1 then AGE1 is 1
egress_df.loc[egress_df['AGE_C']==1,'AGE1']=1
egress_df.loc[egress_df['AGE_C']==2,'AGE2']=1
egress_df.loc[egress_df['AGE_C']==3,'AGE3']=1



In [ ]:
egress_df['RENT_BICYCLE']=egress_df['RENT_BICYCLE'].replace({4: 1, 3: 2})
egress_df['RENT_BICYCLE']=egress_df['RENT_BICYCLE'].replace({2: 3, 1: 2})
egress_df['RENT_BICYCLE']=egress_df['RENT_BICYCLE'].replace({3:1})

egress_df['WORK']=0
egress_df.loc[(egress_df['O_TYPE']=='Work') | (egress_df['D_TYPE']=='Work'),'WORK']=1

egress_df['DEN']=0
egress_df['DEN']=egress_df['INTERSECTIONS']/egress_df['DISTANCE']

egress_df=egress_df[egress_df['DEN']<30]

egress_df['HIGH']=0
egress_df['MED']=0

# if DEN is greater than 66 percentile of data of DEN then HIGH is 1 for that entry
# if DEN is between 33 and 66 percentiles of data of DEN then MED is 1 for that entry
egress_df.loc[egress_df['DEN']>egress_df['DEN'].quantile(0.66),'HIGH']=1
egress_df.loc[(egress_df['DEN']<=egress_df['DEN'].quantile(0.66)) & (egress_df['DEN']>egress_df['DEN'].quantile(0.33)),'MED']=1

egress_df['UNO']=1

egress_df['FAV']=1
egress_df.loc[(egress_df['SLOPE_TYPE']=='rise') | (egress_df['SLOPE_TYPE']=='mixed')| (egress_df['SLOPE_TYPE']=='hydrograph'),'FAV']=0



egress_df[['ID', 'RENT_BICYCLE','MALE', 'SLOPE', 'SLOPE_TYPE', 'DISTANCE', 'AGE1','AGE2','AGE3',
           'TIME','O_TYPE','D_TYPE','EGRESS_MODE','VEH','INTERSECTIONS','WORK','DEN','UNO',
           'UP','DOWN','FLAT','LONG','SHORT','MEDIUM','FAV','HIGH','MED']].to_csv(path+"egress_data_analysis.csv",index=True)

In [ ]:
print(egress_df[egress_df['DISTANCE']<1].shape[0])
print(egress_df[(egress_df['DISTANCE']>=1) & (egress_df['DISTANCE']<2)].shape[0])
print(egress_df[(egress_df['DISTANCE']>=2) & (egress_df['DISTANCE']<3)].shape[0])
print(egress_df[(egress_df['DISTANCE']>=3) & (egress_df['DISTANCE']<4)].shape[0])
print(egress_df[(egress_df['DISTANCE']>=4)].shape[0])

304
169
75
26
81
